In [2]:
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- CDS Pricing Functions ---
def calculate_cds_values(notional, spread_bps, term_years, recovery_rate, interest_rate):
    hazard_rates = np.linspace(0.01, 0.20, 1000)
    spread = spread_bps / 10000  # Convert bps to decimal
    r = interest_rate
    R = recovery_rate
    t = term_years
    N = notional * 1_000_000  # Convert millions to absolute dollars

    premiums, protections, npvs = [], [], []

    for h in hazard_rates:
        discount_factors = np.exp(-r * np.arange(1, t + 1))
        survival_probs = np.exp(-h * np.arange(1, t + 1))

        premium_leg = N * spread * np.sum(discount_factors * survival_probs)
        protection_leg = N * (1 - R) * np.sum(
            h * np.exp(-h * np.arange(1, t + 1)) * discount_factors
        )

        npv = protection_leg - premium_leg

        premiums.append(premium_leg)
        protections.append(protection_leg)
        npvs.append(npv)

    return hazard_rates * 100, premiums, protections, npvs

def find_breakeven_hazard_rate(hazard_rates, npvs):
    npvs = np.array(npvs)
    signs = np.sign(npvs)
    sign_changes = np.where(np.diff(signs))[0]
    if len(sign_changes) == 0:
        return None
    i = sign_changes[0]
    # Linear interpolation for better accuracy
    x0, x1 = hazard_rates[i], hazard_rates[i+1]
    y0, y1 = npvs[i], npvs[i+1]
    if y1 != y0:
        breakeven = x0 - y0 * (x1 - x0) / (y1 - y0)
        return breakeven
    else:
        return x0

# --- UI Elements ---
style = {'description_width': '250px'}
layout = widgets.Layout(width='800px')

notional_input = widgets.FloatText(value=10, description="Notional ($ millions):", style=style, layout=layout)
spread_input = widgets.FloatText(value=150, description="Spread (bps):", style=style, layout=layout)
term_input = widgets.IntText(value=5, description="Term (years):", style=style, layout=layout)
recovery_input = widgets.FloatText(value=40, description="Recovery Rate (%):", style=style, layout=layout)
interest_input = widgets.FloatText(value=3, description="Interest Rate (%):", style=style, layout=layout)
update_button = widgets.Button(description="Update Graph")

output = widgets.Output()

# --- Plotting Function ---
def update_graph(_):
    with output:
        clear_output()

        notional = notional_input.value
        spread = spread_input.value
        term = term_input.value
        recovery_rate = recovery_input.value / 100  # Convert % to decimal
        interest_rate = interest_input.value / 100  # Convert % to decimal

        hazard_rates, premiums, protections, npvs = calculate_cds_values(
            notional, spread, term, recovery_rate, interest_rate
        )

        breakeven = find_breakeven_hazard_rate(hazard_rates, npvs)

        fig = go.Figure()

        # Shaded region for Profit
        fig.add_trace(go.Scatter(
            x=hazard_rates,
            y=[max(n, 0) for n in npvs],
            fill='tozeroy',
            mode='none',
            fillcolor='rgba(0, 200, 0, 0.1)',
            name='Positive NPV',
            hoverinfo='skip'
        ))

        # Shaded region for Loss
        fig.add_trace(go.Scatter(
            x=hazard_rates,
            y=[min(n, 0) for n in npvs],
            fill='tozeroy',
            mode='none',
            fillcolor='rgba(200, 0, 0, 0.1)',
            name='Negative NPV',
            hoverinfo='skip'
        ))

        fig.add_trace(go.Scatter(
            x=hazard_rates,
            y=premiums,
            mode='lines',
            name='Premium Leg PV',
            hovertemplate='Premium Leg PV: $%{y:,.2f}<extra></extra>'
        ))

        fig.add_trace(go.Scatter(
            x=hazard_rates,
            y=protections,
            mode='lines',
            name='Protection Leg PV',
            hovertemplate='Protection Leg PV: $%{y:,.2f}<extra></extra>'
        ))

        fig.add_trace(go.Scatter(
            x=hazard_rates,
            y=npvs,
            mode='lines',
            name='Net Present Value',
            hovertemplate='Net Present Value: $%{y:,.2f}<extra></extra>'
        ))

        if breakeven is not None:
            fig.add_trace(go.Scatter(
                x=[breakeven],
                y=[0],
                mode='markers+text',
                marker=dict(color='black', size=10),
                text=[f'Breakeven: {breakeven:.2f}%'],
                textposition='top right',
                name='Breakeven Point',
                hovertemplate='Breakeven Hazard Rate: %{x:.2f}%<extra></extra>'
            ))

        fig.update_layout(
            title='CDS Valuation vs Hazard Rate',
            xaxis=dict(
                title='Hazard Rate (%)',
                showgrid=True,
                gridwidth=1,
                gridcolor='LightGray',
                zeroline=False,
                showline=True,
                linewidth=1,
                linecolor='Black',
                minor=dict(
                    showgrid=True,
                    gridwidth=0.5,
                    gridcolor='LightGray',
                ),
                dtick=1,
                minor_ticks='outside'
            ),
            yaxis=dict(
                title='Value ($)',
                showgrid=True,
                gridwidth=1,
                gridcolor='LightGray',
                zeroline=False,
                showline=True,
                linewidth=1,
                linecolor='Black',
                minor=dict(
                    showgrid=True,
                    gridwidth=0.5,
                    gridcolor='LightGray',
                ),
                minor_ticks='outside',
            ),
            hovermode='x unified',
            height=600
        )

        display(fig)

update_button.on_click(update_graph)

# --- Display UI ---
display(notional_input, spread_input, term_input, recovery_input, interest_input, update_button, output)

# Run once at start
update_graph(None)


ModuleNotFoundError: No module named 'numpy'